<a href="https://colab.research.google.com/github/arguntolga/ts_jupyter/blob/main/ss_sheets11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade -q gspread
!pip install gspread_formatting

import yfinance as yf
import pandas as pd
import seaborn as sns
import warnings
import gspread
import matplotlib
from google.colab import auth
from google.auth import default
from gspread_formatting import ConditionalFormatRule
from gspread_formatting.models import CellFormat, Color, TextFormat, NumberFormat
from gspread_formatting.functions import format_cell_ranges, set_frozen
from gspread_formatting import set_column_width , set_column_widths
from matplotlib import colors
from matplotlib import pyplot
import numpy as np
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# Define your two stock tickers
#stock_list = ["EREGL.IS", "FROTO.IS", "THYAO.IS", "TTRAK.IS", "MIATK.IS"]
stock_list = [ "XU100.IS",
   "AGROT.IS","AKBNK.IS","AKSA.IS","AKSEN.IS","ALARK.IS","ALFAS.IS","ALGYO.IS",
   "ALKA.IS","ALKIM.IS","ANGEN.IS","ARCLK.IS","ARDYZ.IS","ASELS.IS","ASTOR.IS",
   "BIMAS.IS","BNTAS.IS","BOBET.IS","BRISA.IS","BRYAT.IS","CIMSA.IS","CLEBI.IS",
   "CVKMD.IS","CWENE.IS","DOAS.IS","DOHOL.IS","EGEEN.IS","EKGYO.IS","ELITE.IS",
   "ENJSA.IS","ENKAI.IS","ERBOS.IS","EREGL.IS","EUPWR.IS","FROTO.IS","GARAN.IS",
   "GESAN.IS","GUBRF.IS","GWIND.IS","HALKB.IS","HEKTS.IS","HKTM.IS","HUNER.IS",
   "IPEKE.IS","ISCTR.IS","ISGYO.IS","ISMEN.IS","ISYAT.IS","JANTS.IS","KAREL.IS",
   "KARSN.IS","KCHOL.IS","KLKIM.IS","KLSER.IS","KMPUR.IS","KONTR.IS","KONYA.IS",
   "KORDS.IS","KOZAA.IS","KOZAL.IS","KRDMD.IS","LOGO.IS","MAVI.IS","MGROS.IS",
   "MIATK.IS","ODAS.IS","ORGE.IS","OTKAR.IS","OYAKC.IS","PARSN.IS","PETKM.IS",
   "PGSUS.IS","PNLSN.IS","POLTK.IS","SAHOL.IS","SASA.IS","SAYAS.IS","SISE.IS",
   "SMRTG.IS","SOKM.IS","SUWEN.IS","TABGD.IS","TAVHL.IS","TCELL.IS","TEZOL.IS",
   "THYAO.IS","TKFEN.IS","TKNSA.IS","TOASO.IS","TTKOM.IS","TTRAK.IS","TUKAS.IS",
   "TUPRS.IS","ULKER.IS","ULUUN.IS","VESBE.IS","VESTL.IS","YAPRK.IS","YATAS.IS",
   "YEOTK.IS","YKBNK.IS","YUNSA.IS","YYLGD.IS"

]

# Set start and end dates
start_date = "2023-01-04"
end_date = "2024-12-11"

# Download data using yfinance
data = yf.download(stock_list, start=start_date, end=end_date)["Adj Close"]
data = data.fillna(0)
data.index = data.index.date
#print(data)

def calculate_moving_averages(stock_list, data, window_sizes):
  for stock in stock_list:
    for window in window_sizes:
      data[stock + '_MA_' + str(window)] = data[stock].rolling(window=window).mean()
  return data

def calculate_above_below_ma(stock_list, data, window_sizes):
  above_below_counts = {}
  for date in data.index:
    daily_counts = {"Date": date}
    for window in window_sizes:
      above_count = 0
      for stock in stock_list:
        ma_col = f"{stock}_MA_{window}"
        above_count += (data.loc[date, stock] > data.loc[date, ma_col])
      below_count = len(stock_list) - above_count
      daily_counts["Above MA" + str(window)] = above_count
      daily_counts["Below MA" + str(window)] = below_count
    above_below_counts[date] = daily_counts
  return pd.DataFrame(above_below_counts).transpose()


def calculate_super_stock(stock_list, data, window_sizes):
  super_stock = 0
  super_stock_df = pd.DataFrame(index=data.index, columns=stock_list)
  for date in data.index:
    for stock in stock_list:
      super_stock_level = 0
      if data.loc[date, stock] > data.loc[date, f"{stock}_MA_{window_sizes[0]}"]:
          super_stock_level += 1
      for window_idx in range(len(window_sizes) - 1):
        current_ma_col = f"{stock}_MA_{window_sizes[window_idx]}"
        next_ma_col = f"{stock}_MA_{window_sizes[window_idx + 1]}"
        if data.loc[date, current_ma_col] > data.loc[date, next_ma_col]:
          super_stock_level += 1
      super_stock_df.loc[date, stock] = int(super_stock_level)
  return super_stock_df


# Define window sizes for moving averages
window_sizes = [10, 20, 50, 100, 200]

# Apply the moving average calculation
data_with_ma = calculate_moving_averages(stock_list, data.copy(), window_sizes)
#display(data_with_ma)

# Calculate above/below MA counts
#above_below_data = calculate_above_below_ma(stock_list, data_with_ma.copy(), window_sizes)
#print(above_below_data)

super_stock_data = calculate_super_stock(stock_list, data_with_ma, window_sizes)

#display(above_below_data)
#display(super_stock_data)

super_stock_data_reversed = super_stock_data.iloc[::-1]

def remove_is_suffix(df):
  pattern = r"(.*).IS$"
  new_columns = df.columns.str.replace(pattern, r"\1", regex=True)
  df = df.rename(columns=dict(zip(df.columns, new_columns)))
  return df

super_stock_data_reversed = remove_is_suffix(super_stock_data_reversed.copy())

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open the spreadsheet
spreadsheet_name = 'super_stock1'
spreadsheet = gc.open(spreadsheet_name)

def delete_and_create_sheet(spreadsheet, old_sheet_name, new_sheet_name, number_of_columns):
  # Check if there are more than one sheet before deleting
  if len(spreadsheet.worksheets()) > 1:
    # Delete the existing sheet (if provided)
    if old_sheet_name:
      try:
        worksheet = spreadsheet.worksheet(old_sheet_name)
        spreadsheet.del_worksheet(worksheet)
        print(f"Sheet '{old_sheet_name}' deleted successfully.")
      except gspread.SpreadsheetNotFound:
        print(f"Sheet '{old_sheet_name}' not found. Skipping deletion.")
  else:
    print("Cannot delete the only sheet in the document.")

  # Create a new sheet, specifying number of rows and columns
  new_worksheet = spreadsheet.add_worksheet(title=new_sheet_name, rows=500, cols=number_of_columns) # Added rows and cols
  print(f"Sheet '{new_sheet_name}' created successfully.")

old_sheet_name = 'SuperStock (Daily)'
new_sheet_name = 'SuperStock (Daily)'

number_of_columns = len(super_stock_data_reversed.columns)

delete_and_create_sheet(spreadsheet, old_sheet_name, new_sheet_name, number_of_columns)



def set_sheet_as_first(spreadsheet, target_sheet_name):
  # Get all worksheets
  all_worksheets = spreadsheet.worksheets()

  # Find the target sheet by name
  target_sheet = None
  for sheet in all_worksheets:
    if sheet.title == target_sheet_name:
      target_sheet = sheet
      break

  if not target_sheet:
    print(f"Sheet '{target_sheet_name}' not found.")
    return

  # Reorder sheet list (target sheet first, others follow)
  reordered_sheets = [target_sheet] + [sheet for sheet in all_worksheets if sheet != target_sheet]

  # Update sheet order
  for index, sheet in enumerate(reordered_sheets):
    # Use the 'sheet' object (Worksheet) to update its properties
    sheet.update_index(index) # index starts at 0

  print(f"Sheet '{target_sheet_name}' set as the first sheet successfully.")

set_sheet_as_first(spreadsheet, new_sheet_name)




#worksheet = gc.open("super_stock1")
#sheet1 = worksheet.sheet1
sheet1 = spreadsheet.worksheet(new_sheet_name)

sheet1.update_title("SuperStock (Daily)")

def _to_string(df: pd.DataFrame, *cols) -> pd.DataFrame:
    for col in cols:
        if col in df.columns:
            df[col] = df[col].astype(str)
    return df

super_stock_data_reversed_no_dated = super_stock_data_reversed

super_stock_data_reversed = super_stock_data_reversed.reset_index()
super_stock_data_reversed = super_stock_data_reversed.rename(columns={'index': 'Date'})
super_stock_data_reversed = _to_string(super_stock_data_reversed, "Date")

sheet1.update([super_stock_data_reversed.columns.values.tolist()] + super_stock_data_reversed.fillna(-1).values.tolist())

fmt = CellFormat(
    backgroundColor=Color(red=0.2, green=0.2, blue=0.2, alpha=1),
    textFormat=TextFormat(
        foregroundColor=Color(1, 1, 1),
        fontFamily="Merriweather",
        fontSize=10,
        bold=True
    ),
    horizontalAlignment="CENTER",
)

fmt2 = CellFormat(horizontalAlignment="CENTER")

format_cell_ranges(sheet1, [("A1:ZZ1", fmt), ("A2:ZZ500", fmt2)])
set_frozen(sheet1, rows=1, cols=1)
set_column_widths(sheet1, [('A', 78), ('B:EZ', 56)])


def index_to_spreadsheet_column(column_int, start_index=0):
  # https://stackoverflow.com/a/23862195/171847
  letter = ''
  while column_int > 25 + start_index:
      letter += chr(65 + int((column_int-start_index)/26) - 1)
      column_int = column_int - (int((column_int-start_index)/26))*26
  letter += chr(65 - start_index + (int(column_int)))
  return letter


# https://scipy-cookbook.readthedocs.io/items/Matplotlib_ColormapTransformations.html
def cmap_map(function, cmap):
    """ Applies function (which should operate on vectors of shape 3: [r, g, b]), on colormap cmap.
    This routine will break any discontinuous points in a colormap.
    """
    cdict = cmap._segmentdata
    step_dict = {}
    # Firt get the list of points where the segments start or end
    for key in ('red', 'green', 'blue'):
        step_dict[key] = list(map(lambda x: x[0], cdict[key]))
    step_list = sum(step_dict.values(), [])
    step_list = np.array(list(set(step_list)))
    # Then compute the LUT, and apply the function to the LUT
    reduced_cmap = lambda step : np.array(cmap(step)[0:3])
    old_LUT = np.array(list(map(reduced_cmap, step_list)))
    new_LUT = np.array(list(map(function, old_LUT)))
    # Now try to make a minimal segment definition of the new LUT
    cdict = {}
    for i, key in enumerate(['red','green','blue']):
        this_cdict = {}
        for j, step in enumerate(step_list):
            if step in step_dict[key]:
                this_cdict[step] = new_LUT[j, i]
            elif new_LUT[j,i] != old_LUT[j, i]:
                this_cdict[step] = new_LUT[j, i]
        colorvector = list(map(lambda x: x + (x[1], ), this_cdict.items()))
        colorvector.sort()
        cdict[key] = colorvector

    return matplotlib.colors.LinearSegmentedColormap('colormap',cdict,1024)





#spectral_cmap = pyplot.get_cmap('Spectral')
spectral_cmap = pyplot.get_cmap('YlGn')
light_spectral_cmap = cmap_map(lambda x: x/2 + 0.5, spectral_cmap)

formats = []
for col_index in range(1, len(super_stock_data_reversed.columns)): #len(super_stock_data_reversed.columns)-1):
  col = super_stock_data_reversed.iloc[:, col_index]
  min_value = col.min()
  max_value = col.max()
  norm = colors.Normalize(vmin=min_value, vmax=max_value, clip=True)

  idx = 2

  for value in col:
    normanized_value = norm(value)
    cell_id = f"{index_to_spreadsheet_column(col_index)}{idx}"
    color = light_spectral_cmap(normanized_value)
    formats.append({
        "range": f"{cell_id}:{cell_id}",
        "format": {
            "backgroundColor": {
              "red": color[0],
              "green": color[1],
              "blue": color[2]
            }
        }
    })
    idx += 1

batch_format_response = sheet1.batch_format(formats)



cm = sns.light_palette("green", as_cmap=True)
super_stock_data_reversed_no_dated.apply(pd.to_numeric).style.set_sticky(axis=1).background_gradient(cmap=cm)